In [ ]:
! pip install datasets transformers

  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.9.0-py3-none-any.whl (179 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import transformers

print(transformers.__version__)

4.47.1


In [ ]:
!pip install pyspark datasets pandas

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("QA_Dataset").getOrCreate()

# Load CSV into a Spark DataFrame
df = spark.read.csv("/tmp/temp/output.csv", header=True, inferSchema=True)

# Show the first few rows
df.show()

+--------------------+--------------------+
|            Question|              Answer|
+--------------------+--------------------+
|What is the purpo...| It is designed t...|
|Who is the Recomm...| It is specifical...|
|What technology d...| It uses advanced...|
|What does the Rec...| It provides pers...|
|What are the key ...| Real-time person...|
|How does the Reco...| It delivers dyna...|
|What makes the Re...| It can handle bu...|
|What pricing mode...| It offers flexib...|
|What is the benef...| The engine can r...|
|How does the Reco...| It uses machine ...|
|What impact does ...| Clients see up t...|
|Why is the Recomm...| It is affordable...|
|What is the call ...| Contact the comp...|
|  The eCommend Ne...| open-source reco...|
|  eCommend uses c...| including collab...|
|  Key features in...| powerful persona...|
|  eCommend analyz...| such as browsing...|
|  eCommend is cos...| scalable, and de...|
|  As a cloud-base...| eCommend can sca...|
|  eCommend levera...| seamless 

In [ ]:
pandas_df = df.toPandas()

In [ ]:
from datasets import Dataset

# Convert Pandas DataFrame to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(pandas_df)

# Show a sample
print(hf_dataset[0])

{'Question': 'What is the purpose of the Recommendation Engine?', 'Answer': ' It is designed to boost sales and engagement by providing personalized product and content recommendations.'}


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["Question"], examples["Answer"], padding="max_length", truncation=True)

# Apply tokenization to the dataset
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# Show a sample after tokenization
print(tokenized_dataset[0])


Map:   0%|          | 0/49 [00:00<?, ? examples/s]

{'Question': 'What is the purpose of the Recommendation Engine?', 'Answer': ' It is designed to boost sales and engagement by providing personalized product and content recommendations.', 'input_ids': [101, 2054, 2003, 1996, 3800, 1997, 1996, 12832, 3194, 1029, 102, 2009, 2003, 2881, 2000, 12992, 4341, 1998, 8147, 2011, 4346, 3167, 3550, 4031, 1998, 4180, 11433, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from datasets import DatasetDict

# Split dataset (90% train, 10% validation)
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)

# Define train and validation sets
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(eval_dataset)}")

Training samples: 44, Validation samples: 5


In [ ]:
!pip install transformers datasets torch evaluate

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load model & tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/tmp/temp",          # Directory to save model checkpoints
    evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=2e-5,              # Standard BERT fine-tuning LR
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,              # Adjust based on your dataset size
    weight_decay=0.01,               # Regularization
    logging_dir="/tmp/temp",            # Logs for TensorBoard
    logging_steps=10,
    save_strategy="epoch",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

<ipython-input-13-dda2bb812d2b>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ValueError: The model did not return a loss from the inputs, only the following keys: start_logits,end_logits. For reference, the inputs it received are input_ids,token_type_ids,attention_mask.

In [ ]:
def add_token_positions(example):
    inputs = tokenizer(example["Question"], example["Answer"], truncation=True, padding="max_length")

    # Find start and end positions
    start_position = inputs.input_ids.index(tokenizer.cls_token_id) + 1  # Example heuristic
    end_position = start_position + len(tokenizer.tokenize(example["Answer"])) - 1

    example["input_ids"] = inputs["input_ids"]
    example["attention_mask"] = inputs["attention_mask"]
    example["token_type_ids"] = inputs["token_type_ids"]
    example["start_positions"] = start_position
    example["end_positions"] = end_position

    return example

# Apply function
tokenized_dataset = hf_dataset.map(add_token_positions, batched=False)

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=None,  # Let Hugging Face handle batch collation
    compute_metrics=None,  # No standard accuracy metric for QA, so keep None for now
)

<ipython-input-18-58f6bfd21c52>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,No log
2,5.501100,No log
3,4.050500,No log


TrainOutput(global_step=21, training_loss=4.752235685076032, metrics={'train_runtime': 631.558, 'train_samples_per_second': 0.233, 'train_steps_per_second': 0.033, 'total_flos': 38410623240192.0, 'train_loss': 4.752235685076032, 'epoch': 3.0})

In [ ]:
from shutil import make_archive, copytree

# Rename the model directory for clarity (optional)
copytree("/tmp/temp", "./qa_model")

# Compress the model folder
make_archive("qa_model", 'zip', "./qa_model")

# Download the zip file (if using Google Colab)
from google.colab import files
files.download("qa_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# Load fine-tuned model
model = AutoModelForQuestionAnswering.from_pretrained("/tmp/temp/checkpoint-21")
tokenizer = AutoTokenizer.from_pretrained("/tmp/temp/checkpoint-21")

# Push to Hugging Face Hub (replace "your-username" with your actual username)
model.push_to_hub("atekrugis/bert_uncased_qa_model")
tokenizer.push_to_hub("atekrugis/bert_uncased_qa_model")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/atekrugis/bert_uncased_qa_model/commit/a63d2e3581665bca3351aafe8e197617c96d0f9e', commit_message='Upload tokenizer', commit_description='', oid='a63d2e3581665bca3351aafe8e197617c96d0f9e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/atekrugis/bert_uncased_qa_model', endpoint='https://huggingface.co', repo_type='model', repo_id='atekrugis/bert_uncased_qa_model'), pr_revision=None, pr_num=None)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="atekrugis/bert_uncased_qa_model")

test_data = [
    {"question": "Where is the Eiffel Tower?", "context": "The Eiffel Tower is in Paris, France."},
    {"question": "When was the Eiffel Tower built?", "context": "It was constructed in 1889."}
]

# Evaluate predictions
for item in test_data:
    result = qa_pipeline(item)
    print(f"Question: {item['question']}")
    print(f"Answer: {result['answer']}")

Device set to use cpu


Question: Where is the Eiffel Tower?
Answer: The Eiffel
Question: When was the Eiffel Tower built?
Answer: constructed in 1889


In [ ]:
from transformers import Trainer, TrainingArguments

# Define evaluation arguments (these can be customized)
training_args = TrainingArguments(
    output_dir="/tmp/temp",
    evaluation_strategy="epoch",  # Evaluate every epoch
    per_device_eval_batch_size=8,
    logging_dir="/tmp/logs",
)

# Initialize the Trainer with evaluation
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Add metrics function
)

# Start evaluation
results = trainer.evaluate()

print("Evaluation Results:", results)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


NameError: name 'compute_metrics' is not defined

In [ ]:
!pip install evaluate

In [ ]:
from evaluate import load

# Load the SQuAD metric
squad_metric = load("squad")

def compute_metrics(pred):
    """
    Computes the evaluation metrics for the QA model.

    Parameters:
    - pred: The model's predictions.

    Returns:
    - Dictionary with computed Exact Match (EM) and F1 Score.
    """
    predictions, references = pred
    return squad_metric.compute(predictions=predictions, references=references)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/tmp/temp",
    evaluation_strategy="epoch",  # Evaluate after every epoch
    per_device_eval_batch_size=8,
    logging_dir="/tmp/logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # ✅ Now it should work
)

# Run evaluation
results = trainer.evaluate()

print("Evaluation Results:", results)

<ipython-input-29-8d5edc76e0d7>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation Results: {'eval_model_preparation_time': 0.0061, 'eval_runtime': 10.5133, 'eval_samples_per_second': 0.476, 'eval_steps_per_second': 0.095}


In [ ]:
from evaluate import load
import numpy as np

# Load Hugging Face's metric evaluation module
precision_metric = load("precision")
recall_metric = load("recall")
f1_metric = load("f1")

def compute_metrics(p):
    """
    Computes Precision, Recall, and F1 Score for the QA model.

    Parameters:
    - p: A tuple containing (predictions, labels).

    Returns:
    - Dictionary with computed metrics.
    """
    predictions, references = p

    # Compute individual metrics
    precision = precision_metric.compute(predictions=predictions, references=references)["precision"]
    recall = recall_metric.compute(predictions=predictions, references=references)["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=references)["f1"]

    return {"precision": precision, "recall": recall, "f1": f1}

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/tmp/temp/checkpoint-21",
    evaluation_strategy="epoch",  # Evaluate every epoch
    per_device_eval_batch_size=8,
    logging_dir="/tmp/logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # ✅ Include the function here
)

# Run evaluation
results = trainer.evaluate()

print("Evaluation Results:", results)

<ipython-input-32-595f84708181>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Evaluation Results: {'eval_model_preparation_time': 0.0059, 'eval_runtime': 10.9154, 'eval_samples_per_second': 0.458, 'eval_steps_per_second': 0.092}


In [ ]:
pip install scikit-learn

In [ ]:
print(eval_dataset[0])  # Print the first sample to see its structure


{'Question': 'What does the Recommendation Engine provide?', 'Answer': ' It provides personalized recommendations for products, services, or content in real-time.', 'input_ids': [101, 2054, 2515, 1996, 12832, 3194, 3073, 1029, 102, 2009, 3640, 3167, 3550, 11433, 2005, 3688, 1010, 2578, 1010, 2030, 4180, 1999, 2613, 1011, 2051, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 